In [1]:
import albumentations as A
import cv2
import json
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.image_augmentation import get_bbox_from_json, plot_img_with_bbox, transformed

Transform will be applied on images and related bounding boxes.

In [3]:
transform = A.Compose(
    [
        A.Resize(height=480, width=640),
        A.RandomCrop(width=450, height=450),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
    ],
    bbox_params=A.BboxParams(format="coco"),
)

In [4]:
smartphone = {
    "path_image": "../data/raw/smartphone",
    "path_json": "../data/labeling/smartphone_labeling.json",
}

nano = {
    "path_image": "../data/raw/nano",
    "path_json": "../data/labeling/nano_labeling.json",
}

In [5]:
result = transformed(smartphone, transform)

In [6]:
for i in range(0, len(result)):
    plot_img_with_bbox(
        transformed_image=result[i]["image"],
        transformed_bboxes=result[i]["bboxes"],
    )